 - __This dataset containing Daily Closing Prices for the SPY going from 1986 to 2018__
 - __The idea is to take 30 continuous sequences and predict for the next periods__
 - __Primarily to detect anomalies based on threshold criteria__

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
url = "https://raw.githubusercontent.com/arimitramaiti/datasets/master/LSTM/spx.csv"
dataset = pd.read_csv(url, error_bad_lines=False, header=0, parse_dates=["date"], index_col="date")
dataset.head(4)
dataset.isnull().sum()

close    0
dtype: int64

In [3]:
size = int(dataset.shape[0] * 0.95)
train = dataset.iloc[:size, :]
test = dataset.iloc[size: , :]
train.shape, test.shape

((7782, 1), (410, 1))

Import scaler modules. Expermientally MinMax and Standard could not bring RMSE down, hence used Robust method

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

 - Function to create sequence of inputs specific to LSTM network
 - At each iteration X will have first n number of rows (as per n_steps), y data will have the nth data point as dependent attribute

In [6]:
def lstm_inputs(X_data, y_data, n_steps):
    X, y = [], []
    X_data = pd.DataFrame(X_data)
    y_data = pd.DataFrame(y_data)
    for i in range(len(X_data) - n_steps):
        X_rows = X_data.iloc[i:(i + n_steps)].values
        X.append(X_rows)
        y.append(y_data.iloc[i + n_steps])
    return np.array(X), np.array(y)

Call the function created above first on training inputs and output, then on testing input and output. The order does not matter though

In [7]:
X_train, y_train = lstm_inputs(train, train, 30)
X_test, y_test = lstm_inputs(test, test, 30)

print(X_train.shape, y_train.shape, X_test.shape, X_test.shape)

(7752, 30, 1) (7752, 1) (380, 30, 1) (380, 30, 1)


Import keras and keras modules from keras.layers required for LSTM network

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [22]:
lstm = keras.Sequential()

lstm.add(LSTM(units=64, activation="tanh", input_shape=(30, 1)))

lstm.add(Dropout(0.20))

lstm.add(RepeatVector(n=30))

lstm.add(LSTM(units=32, return_sequences=True))

lstm.add(Dropout(0.20))

lstm.add(TimeDistributed(Dense(y_train.shape[1], activation='softmax')))

lstm.compile(optimizer="adam", loss="mae")

In [11]:
# lstm.summary()

In [23]:
history = lstm.fit(X_train, y_train, epochs=10, batch_size=32, shuffle=False, validation_split=0.10)

ValueError: Error when checking target: expected time_distributed_5 to have 3 dimensions, but got array with shape (7752, 1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Train vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

In [ ]:
y_pred = lstm.predict(X_test)

In [ ]:
y_train_inv = mx.inverse_transform(y_train.reshape(1, -1))
y_test_inv = mx.inverse_transform(y_test.reshape(1, -1))
y_pred_inv = mx.inverse_transform(y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test_inv.flatten(), y_pred_inv.flatten(), squared=False)

In [ ]:
y_pred_inv.flatten().std()

https://github.com/keras-team/keras/issues/5214

 - __Somehow the input dimensions is not getting accepted by TimeDistributed layer__
 - __Need to try on a different case study__
 - __Sought help from Stackoverflow and venelin@curiousily.com__